# Advent of Code 2021 - Day 9

In [29]:
typealias Index = Pair<Int, Int>
typealias HeightMap = List<List<Int>>

In [30]:
import java.io.File
import java.util.Scanner

val heightMap: HeightMap = File("Day9.input.txt")
    .bufferedReader()
    .readLines()
    .map { line -> line.map { it.digitToInt() } }

## Part 1

Find the low points in the `heightMap`. A low point is a location that is lower than any of its adjacent (up, down, left, right) locations. A low point's risk level is its height plus one. Calculate the sum risk levels of all low points.

In [31]:
fun HeightMap.left(index: Index): Boolean = index.second - 1 < 0 || 
    this[index.first][index.second - 1] > this[index.first][index.second]

fun HeightMap.right(index: Index): Boolean = index.second + 1 >= this[index.first].size || 
    this[index.first][index.second + 1] > this[index.first][index.second]

fun HeightMap.up(index: Index): Boolean = index.first - 1 < 0 || 
    this[index.first - 1][index.second] > this[index.first][index.second]

fun HeightMap.down(index: Index): Boolean = index.first + 1 >= this.size || 
    this[index.first + 1][index.second] > this[index.first][index.second]

fun <A, B> List<List<A>>.fold2D(accumulator: B, fn: (Index, B, A) -> B) = foldIndexed(accumulator) { rowIndex, acc, row ->
    row.foldIndexed(acc) { colIndex, innerAcc, value -> fn((rowIndex to colIndex), innerAcc, value) }
}

heightMap.fold2D(0) { index, acc, _ -> 
    if (heightMap.left(index) && heightMap.right(index) && heightMap.up(index) && heightMap.down(index)) 
        acc + heightMap[index.first][index.second] + 1
    else acc
}

585

### Notes

Fairly straightforward since the rules for checking whether a location is a low point is not complex. Basically we iterate through each location and check whether the current location is lower than its adjacent locations, if it is, add its value plus one.

A optimization could be made to skip points that have already been checked, since if an element is a low point, its adjacent elements can no longer be low points. The checks are really quick though since it's just very simple if statements.

## Part 2

Find all the basins within the `heightMap`. A basin is a group of locations that eventually flow downwards to a single low point. Locations with height 9 do not count as being in a basin. The size of a basin is the number of locations within it. Find the three largest basins and multiply their sizes together.

In [32]:
class BasinMap(val heightMap: HeightMap) {
    val basins: List<Int>
        get() = heightMap.fold2D(mutableListOf<Int>()) { index, acc, _ ->
            if (!index.visited()) acc.add(index.spread())
            acc
        }

    private val visited: MutableSet<Index> = mutableSetOf()
        
    private fun Index.spread(): Int {
        visited.add(this)
        return 1 + spreadTo { right() } + spreadTo { left() } + spreadTo { down() } + spreadTo { up() }
    }

    private fun Index.spreadTo(indexFn: Index.() -> Index): Int = indexFn().let { if (it.valid() && !it.visited()) it.spread() else 0 }

    private fun Index.right(): Index = (first to second + 1)
    private fun Index.left(): Index = (first to second - 1)
    private fun Index.down(): Index = (first + 1 to second)
    private fun Index.up(): Index = (first - 1 to second)
    private fun Index.valid(): Boolean = second >= 0 && first >= 0 && first < heightMap.size && second < heightMap[first].size
    private fun Index.visited(): Boolean = valid() && (visited.contains(this) || heightMap[first][second] == 9)
}

BasinMap(heightMap)
    .basins
    .sortedDescending()
    .take(3)
    .reduce { acc, it -> acc * it }

827904

### Notes

This one relies on the idea that 9 can never be part of a basin, so it acts as a natural barrier between basins. So rather than checking increasing / decreasing, you can just "spread out" until you encounter a 9. So the general idea is to check each element if it's been "visited", if it hasn't, then spread out in every direction. Mark each visit along the way from that element until you reach a 9. 

An optimization could be made here to keep track of the top 3 largest basins as you go, so that way you don't have to sort at the end, but just returning all of the basins from the calculation is more "flexible." You technically also don't need to sort and can grab the top 3 linearly, but sorting it was an "out of the box" solution.